## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-04-16-54-15 +0000,nypost,"Missing 9-year-old girl found dead, adoptive m...",https://nypost.com/2025/09/04/us-news/missing-...
1,2025-09-04-16-52-00 +0000,wsj,Macron Says 26 Countries Agree to Contribute t...,https://www.wsj.com/world/europe/macron-says-d...
2,2025-09-04-16-50-14 +0000,nyt,"Worst Rains in Decades Batter North India, Sub...",https://www.nytimes.com/2025/09/04/world/asia/...
3,2025-09-04-16-44-08 +0000,bbc,PM refuses to say if he will sack Rayner if sh...,https://www.bbc.com/news/articles/ce321d2n45vo...
4,2025-09-04-16-40-00 +0000,wsj,Auto & Transport Roundup: Market Talk,https://www.wsj.com/business/auto-transport-ro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2298/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
44,trump,63
57,new,14
27,he,13
232,up,12
476,tariffs,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
44,2025-09-04-15-51-49 +0000,nypost,Trump sets deadline to thin field in crowded N...,https://nypost.com/2025/09/04/us-news/trump-se...,135
24,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...,134
219,2025-09-04-03-23-00 +0000,wsj,The Trump administration asked the Supreme Cou...,https://www.wsj.com/politics/policy/trump-admi...,127
315,2025-09-03-19-50-17 +0000,nypost,Trump rips reporter for saying he’s taken ‘no ...,https://nypost.com/2025/09/03/us-news/trump-ri...,126
221,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...,125


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
44,135,2025-09-04-15-51-49 +0000,nypost,Trump sets deadline to thin field in crowded N...,https://nypost.com/2025/09/04/us-news/trump-se...
24,71,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
8,63,2025-09-04-16-36-57 +0000,nypost,CBS News staffers ‘freaking out’ as new Paramo...,https://nypost.com/2025/09/04/media/paramount-...
221,62,2025-09-04-02-55-30 +0000,nypost,Trump admin demands Supreme Court rule quickly...,https://nypost.com/2025/09/03/us-news/trump-ad...
270,54,2025-09-03-22-33-42 +0000,nypost,JD and Usha Vance pay their respects to victim...,https://nypost.com/2025/09/03/us-news/jd-and-u...
77,48,2025-09-04-14-40-18 +0000,nyt,Portugal Searches for Cause of Lisbon Funicula...,https://www.nytimes.com/2025/09/04/world/europ...
308,47,2025-09-03-20-07-05 +0000,nypost,Putin says Zelensky should come to Moscow— as ...,https://nypost.com/2025/09/03/world-news/putin...
12,42,2025-09-04-16-28-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
291,41,2025-09-03-21-02-22 +0000,nypost,China’s innovation speeds ahead while America ...,https://nypost.com/2025/09/03/world-news/china...
6,40,2025-09-04-16-39-01 +0000,bbc,Washington DC sues Trump administration over N...,https://www.bbc.com/news/articles/cvg0eqw9pl7o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
